In [19]:
import mlflow.pytorch
import torch
from torch_geometric.loader import DataLoader
from torch_geometric.nn import global_mean_pool
from torch_geometric.nn.models import DeepGraphInfomax

import graph_reinforcement_learning_using_blockchain_data as grl
from graph_reinforcement_learning_using_blockchain_data import config

config.load_dotenv()

True

In [20]:
dataset = torch.load(config.FLASHBOTS_Q2_DATA_DIR / "state_graphs.pt", weights_only=False)

In [21]:
max_feats_len = 0
for vals in dataset.values():
    for graphs in vals:
        if graphs.x.shape[1] > max_feats_len:
            max_feats_len = graphs.x.shape[1]

In [22]:
for vals in dataset.values():
    for graphs in vals:
        graphs.x = grl.pad_features(graphs.x, max_feats_len)

In [23]:
data_list = [d for sublist in dataset.values() for d in sublist]

generator = torch.Generator().manual_seed(42)
train_dataset, test_dataset = torch.utils.data.random_split(
    data_list, [0.8, 0.2], generator=generator
)

In [24]:
train_loader = DataLoader(
    train_dataset, batch_size=256, shuffle=True, exclude_keys=["account_mapping"], drop_last=False
)
test_loader = DataLoader(
    test_dataset, batch_size=128, shuffle=False, exclude_keys=["account_mapping"], drop_last=False
)
data_loader = DataLoader(
    data_list, batch_size=256, shuffle=False, exclude_keys=["account_mapping"], drop_last=False
)

In [7]:
summary = lambda z, *_, batch=None, **__: global_mean_pool(z, batch)


def corruption(data):
    """
    Corrupts the graph data by shuffling node features.
    Accepts a Data/Batch object and returns a corrupted Data/Batch object.
    """
    corrupted_data = data.clone()  # Create a copy to avoid modifying the original data
    num_nodes_total = data.x.size(0)
    # Generate a permutation of node indices within the batch
    perm = torch.randperm(num_nodes_total, device=data.x.device)
    corrupted_data.x = data.x[perm]  # Apply the permutation to shuffle features
    return corrupted_data


In [8]:
num_node_features = max_feats_len
dim_global_features = 0
hidden_channels = 256  # adjust as needed
num_classes = 2  # binary classification

device = torch.device("mps")
enc = grl.SAGEEncoder(num_node_features, hidden_channels).to(device)
dgi = DeepGraphInfomax(hidden_channels=hidden_channels,
                       encoder=enc,
                       summary=summary,
                       corruption=corruption).to(device)

In [10]:
enc = grl.pretrain_dgi(train_loader, dgi, device, epochs=10)

2025/04/30 14:44:00 WARNING mlflow.utils.autologging_utils: MLflow pytorch autologging is known to be compatible with 1.9.0 <= torch <= 2.5.1, but the installed version is 2.6.0. If you encounter errors during autologging, try upgrading / downgrading torch to a compatible version, or try upgrading MLflow.
 10%|█         | 1/10 [01:25<12:45, 85.02s/it]

[DGI] epoch 01  loss=0.2504


 10%|█         | 1/10 [01:28<13:18, 88.68s/it]

🏃 View run bouncy-goose-341 at: http://127.0.0.1:8080/#/experiments/330930495026013213/runs/9a681177d5384a12b47d0ea30aa45a56
🧪 View experiment at: http://127.0.0.1:8080/#/experiments/330930495026013213


KeyboardInterrupt: 

In [16]:
model_uri = "mlflow-artifacts:/330930495026013213/1f3f3d97f6a14569b9f2f285c5a98bb4/artifacts/model"
enc = mlflow.pytorch.load_model(model_uri).encoder

In [17]:
for p in enc.parameters():
    p.requires_grad = False

model = grl.GraphSAGEClassifier(enc, hidden=256, num_classes=2).to(device)
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(
    filter(lambda p: p.requires_grad, model.parameters()), lr=1e-3
)

In [18]:
model, embeddings = grl.run_experiment("Unsupervised GraphSAGE with DGI", 10, model, train_loader, test_loader,
                                       optimizer, criterion, device, return_embeddings=True)

2025/04/30 14:59:53 WARNING mlflow.utils.autologging_utils: MLflow pytorch autologging is known to be compatible with 1.9.0 <= torch <= 2.5.1, but the installed version is 2.6.0. If you encounter errors during autologging, try upgrading / downgrading torch to a compatible version, or try upgrading MLflow.
  0%|          | 0/10 [00:00<?, ?it/s]

Epoch 1 starts


 10%|█         | 1/10 [02:41<24:11, 161.26s/it]

Epoch: 001, Train Loss: 0.1362, Test Acc: 0.9722
Epoch 2 starts


 20%|██        | 2/10 [04:56<19:27, 145.91s/it]

...

Epoch: 009, Train Loss: 0.0702, Test Acc: 0.9775
Epoch 10 starts


100%|██████████| 10/10 [26:25<00:00, 158.59s/it]

Epoch: 010, Train Loss: 0.0697, Test Acc: 0.9786



2025/04/30 15:26:26 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run agreeable-snipe-205 at: http://127.0.0.1:8080/#/experiments/132032870842317128/runs/ba4bedca3761489498f42f57c3986752
🧪 View experiment at: http://127.0.0.1:8080/#/experiments/132032870842317128
